<a href="https://colab.research.google.com/github/halkirisame/voice-changer/blob/master/Scylla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1.最初に実行

!git clone https://github.com/prism4291/ndrf_dataset
!pip install qrcode[pil]
!wget https://github.com/multitheftauto/unifont/releases/download/v16.0.03/unifont-16.0.03.ttf
import hashlib
import sqlite3
import multiprocessing
import os
import io
import json
import base64
import traceback
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont
import qrcode
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from google.colab import output
name_data=[]
with open("ndrf_dataset/names.txt") as f:
  for l in f:
    l=l.strip()
    if not l:
      continue
    name_data.append(l)
antenna_data=["アンテナなし","ひのたま","とがったこおり","つむじかぜ","らくせき","せいでんき","みずでっぽう","よわいこうせん","おどかす","ちょっとかいふく","しびれとる","めざめる",
  "すこしよわくなれ","すこしやわくなれ","すこしおそくなれ","みんなすこしよけにくい","すこしつよくなれ","すこしかたくなれ","すこしはやくなれ","すこしかわしやすい","げどく",
  "どくになれ","すこしねむらせる","すこししびれさせる","すこしめかくし","ぞくせいなおす",
  "ばくはつ","ロックアイス","ビルかぜ","いしつぶて","いなづま","バケツのみず","ちょっとふっかつ",
  "すこしむてき","すこしこうふん","たくわえる","ぜんぶなおす","ちょっとあしもとガード","スポットライト","ダークボール","ノックダウン"]
body_data=["1A(0最速)","1B(0準速)","1C(0中間)","1D(0準大)","1E(0最大)",
  "2A(3+最速)","2B(3+準速)","2C(3+中間)","2D(3+準大)","2E(3+最大)",
  "3A(3-最速)","3B(3-準速)","3C(3-中間)","3D(3-準大)","3E(3-最大)",
  "4A(6+最速)","4B(6+準速)","4C(6+中間)","4D(6+準大)","4E(6+最大)",
  "5A(10最速)","5B(6-準速)","5C(6-中間)","5D(6-準大)","5E(6-最大)",
  "6A(15最速)","6B(15準速)","6C(15中間)","6D(10準大)","6E(10最大)"]
color_data=["黒色","赤色","水色","緑色","橙色","黄色","青色","白色","紫色"]
depth_data=["","濃","薄"]
head_data=["丸", "横楕円", "縦楕円", "三角", "四角", "石鹸"]
pair_data=["なし","双子","親子"]
name_map={name_data[i]:i for i in range(len(name_data))}
antenna_map={antenna_data[i]:i for i in range(len(antenna_data))}
body_map={body_data[i][0:2]:i for i in range(len(body_data))}
color_map={color_data[i]:i for i in range(len(color_data))}
depth_map={"通常":0,"濃":1,"薄":2}
head_map={head_data[i]:i for i in range(len(head_data))}
pair_map={"なし":0,"双子":1,"親子":2}

def get_chara_data(a,b):
  hash_object = hashlib.sha256()
  hash_object.update(bytes.fromhex("8f5152"+a))
  c=int.from_bytes(bytes.fromhex(hash_object.hexdigest()),byteorder='little',signed=False)
  p=c>>249
  data=[]
  for j in range(2 if p>=125 else 1):
    d=0 if c&0x7f<=75 else 1 if c&0x7f<=89 else 2
    rsid=b+"_"*j
    name=((c>>14&0xfff)+j*p)%[1190,884,702][d]+[0,1190,2074][d]
    antenna=[[0,0,0,0,0,0,0,0,0,0,0],
      [1,2,3,4,5,6,7,8,9,10,11],
      [12,13,14,15,16,17,18,19,20,0,0],
      [21,22,23,24,22,23,24,25,0,0,0],
      [26,27,28,29,30,31,32,0,0,0,0],
      [33,34,35,36,37,38,39,40,0,0,0]
      ][0 if c>>64&0x3ff<=511 else 1 if c>>64&0x3ff<=840 else 2 if c>>64&0x3ff<=970 else 3 if c>>64&0x3ff<=1018 else 4 if c>>64&0x3ff<=1021 else 5
      ][(c>>74&0xfff)%11];
    body=([[6,7,8,11,12,13,16,17,18,21,22,23],[1,2,3,5,9,10,14,15,19,20,24,26,27,28],[0,4,25,29]
      ][0 if c>>105&0x7f<=102 else 1 if c>>105&0x7f<=125 else 2][(c>>112&0x7f)%(12 if c>>105&0x7f<=102 else 14 if c>>105&0x7f<=125 else 4)
      ]+(0 if p<=125 else j*15))%30
    k=antenna if antenna<=8 else antenna-25 if 26<=antenna<=31 else antenna-31 if 38<=antenna<=39 else 0
    color=((c>>142&0x3f)%8+(1 if (c>>142&0x3f)%8>=k else 0)) if 3960<=c>>128&0x3fff<=16095 else k
    depth=0 if c>>128&0x3fff<=16095 else (c>>142&0x3f)%2+1
    head=(c>>155&0xff)%6
    pair=0 if p<=124 else 1 if p<=125 else 2
    data.append([rsid,b,name,antenna,body,color,depth,head,pair])
  return data
def run(i):
  data=[]
  with open("ndrf_dataset/data/{:02x}.txt".format(i)) as f:
    for l in f:
      l=l.strip()
      if not l:
        continue
      a,b=l.split(" ")
      data.extend(get_chara_data(a,b))
  return data
def scylla(db_path, table_name,rsid=None, name=None, antenna=None, body=None, color=None, depth=None, head=None, pair=None):
  results = []
  db = sqlite3.connect(db_path)
  cursor = db.cursor()
  sql = f"SELECT rsid, qr, name, antenna, body, color, depth, head, pair FROM {table_name}"
  conditions = []
  params = []
  if rsid is not None:
    conditions.append("rsid = ?")
    params.append(rsid)
  if name is not None:
    conditions.append("name = ?")
    params.append(name_map[name])
  if antenna is not None:
    conditions.append("antenna = ?")
    params.append(antenna_map[antenna])
  if body is not None:
    conditions.append("body = ?")
    params.append(body_map[body])
  if color is not None:
    conditions.append("color = ?")
    params.append(color_map[color])
  if depth is not None:
    conditions.append("depth = ?")
    params.append(depth_map[depth])
  if head is not None:
    conditions.append("head = ?")
    params.append(head_map[head])
  if pair is not None:
    conditions.append("pair = ?")
    params.append(pair_map[pair])
  if conditions:
    sql += " WHERE " + " AND ".join(conditions)
  cursor.execute(sql, tuple(params))
  results = cursor.fetchall()
  if db:
    db.close()
  return results
def print_results(results):
  if not results:
    print("該当する個体は見つかりませんでした")
    return
  print("-" * 100)
  print("RSID    QR      名前　　　　　　アンテナ　　　　　　　　体格        色　　　頭の形　ペア　")
  print("-" * 100)
  for row in results:
    rsid_val, qr_val, name_val, antenna_val, body_val, color_val, depth_val, head_val, pair_val = row
    rsid_str = rsid_val
    qr_str = qr_val
    name_str = name_data[name_val]
    antenna_str = antenna_data[antenna_val]
    body_str = body_data[body_val]
    color_depth_str = depth_data[depth_val]+color_data[color_val]
    head_str = head_data[head_val]
    pair_str = pair_data[pair_val]
    rsid_str=rsid_str+" "*(8-len(rsid_str))
    qr_str=qr_str+" "*(8-len(qr_str))
    name_str=name_str+"　"*(8-len(name_str))
    antenna_str=antenna_str+"　"*(12-len(antenna_str))
    body_str=body_str+" "*(10-len(body_str))
    color_depth_str=color_depth_str+"　"*(4-len(color_depth_str))
    head_str=head_str+"　"*(4-len(head_str))
    pair_str=pair_str+"　"*(4-len(pair_str))
    print(f"{rsid_str}{qr_str}{name_str}{antenna_str}{body_str}{color_depth_str}{head_str}{pair_str}")
  print("-" * 100)
  print(f"{len(results)}体見つかりました")
db_path = "scylla.db"
table_name = "characters"
font = ImageFont.truetype("unifont-16.0.03.ttf", 16)

def generate_qr(rsid,qr_str,qr_title,qr_color):
  qr = qrcode.QRCode(
      version=1,
      error_correction=qrcode.constants.ERROR_CORRECT_M,
      box_size=16,
      border=4,
  )
  qr.add_data(qr_str)
  qr.make()
  img = qr.make_image(fill_color="#1e1f22", back_color="#ffffff")
  draw = ImageDraw.Draw(img)
  c=["#474747","#ec413a","#35d0e0","#7bde3a","#e16223","#eeec44","#384bea","#fcfcdf","#9e6cf0"][qr_color]
  draw.rectangle([(0,0),img.size], outline=c, width=40)
  draw.rounded_rectangle([(28,28),(img.size[0]-28,img.size[1]-28)],radius=28,outline="#ffffff",width=28)
  draw.text((37, 37), qr_title, fill="#ccbec4", font=font)
  draw.text((37, 36), qr_title, fill="#bcced4", font=font)
  draw.text((36, 36), qr_title, fill="#1e1f22", font=font)
  draw.text((img.size[0]-187, img.size[1]-51), f"{rsid} / Scylla", fill="#ccbec4", font=font)
  draw.text((img.size[0]-187, img.size[1]-52), f"{rsid} / Scylla", fill="#bcced4", font=font)
  draw.text((img.size[0]-188, img.size[1]-52), f"{rsid} / Scylla", fill="#1e1f22", font=font)
  return img
def generate_qr_base64(img):
  buffer = io.BytesIO()
  img.save(buffer, format="PNG")
  img_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
  return f"data:image/png;base64,{img_str}"
def generate_qr_for_frontend(rsid, qr_str, name_s, clr_s, body_s, ant_s, color_idx):
  return generate_qr_base64(generate_qr(rsid, qr_str, f"{name_s} {clr_s} {body_s} {ant_s}", color_idx))
output.register_callback('generate_qr_for_frontend', generate_qr_for_frontend)
def print_html(results):
  table_html = f"""
<style>
.qr-overlay {{
  position: fixed;
  top: 0;
  left: 0;
  width: 100%;
  height: 100%;
  background-color: rgba(0, 0, 0, 0.9);
  display: none;
  justify-content: center;
  align-items: center;
  z-index: 1000;
  cursor: pointer;
}}
.qr-overlay img {{
  max-width: 90%;
  max-height: 90%;
  object-fit: contain;
  border: 5px solid white;
  background-color: white;
  cursor: default;
  box-shadow: 0 0 15px rgba(0,0,0,0.5);
}}

table.sr {{
  border-collapse: collapse;
  width: 100%;
  font-size: 14px;
}}
.sr th, .sr td {{
  border: 1px solid #ddd;
  padding: 5px;
  text-align: left;
  vertical-align: middle;
  overflow: hidden;
  white-space: nowrap;
  text-overflow: ellipsis;
}}
.sr .qr-actions-cell {{
  width: 96px;
  text-align: center;
  padding: 2px;
  vertical-align: middle;
}}
.sr .qr-action-container {{
  display: flex;
  align-items: center;
  justify-content: center;
  gap: 4px;
}}
.sr .button-stack {{
  display: flex;
  flex-direction: column;
  align-items: stretch;
  gap: 2px;
}}
.sr .qr-thumb {{
  width: 36px;
  height: 36px;
  cursor: pointer;
  border: 1px solid #eee;
  object-fit: contain;
  flex-shrink: 0;
}}
.sr .copy-btn,
.sr .save-btn {{
  font-size: 10px;
  padding: 1px 3px;
  cursor: pointer;
  border: 1px solid #ccc;
  background-color: #f8f8f8;
  border-radius: 3px;
  line-height: 1.2;
  text-align: center;
}}
.sr .copy-btn:hover,.sr .save-btn:hover {{ background-color: #e0e0e0; }}
.sr .copy-btn:disabled,.sr .save-btn:disabled {{ background-color: #eee; cursor: not-allowed; }}

</style>

<table class='sr'>
  <thead>
    <tr>
      <th class='qr-actions-cell'>QR</th>
      <th class='rc'>RSID</th>
      <th class='nc'>名前</th>
      <th class='ac'>アンテナ</th>
      <th>体格</th>
      <th>色</th>
      <th>頭の形</th>
      <th>ペア</th>
    </tr>
  </thead>
  <tbody>
"""
  for index, row in enumerate(results):
    rsid, qr_str, name_idx, antenna_idx, body_idx, color_idx, depth_idx, head_idx, pair_idx = row
    try: name_s = name_data[name_idx]
    except (IndexError, TypeError): name_s = f"不明:{name_idx}"
    try: ant_s = antenna_data[antenna_idx]
    except (IndexError, TypeError): ant_s = f"不明:{antenna_idx}"
    try: body_s = body_data[body_idx]
    except (IndexError, TypeError): body_s = f"不明:{body_idx}"
    clr_s = "不明"
    try:
        depth_str = depth_data[depth_idx] if depth_idx is not None and 0 <= depth_idx < len(depth_data) else ""
        color_str = color_data[color_idx] if color_idx is not None and 0 <= color_idx < len(color_data) else f"不明:{color_idx}"
        clr_s = f"{depth_str}{color_str}"
    except (IndexError, TypeError): clr_s = f"不明:{color_idx},{depth_idx}"
    try: head_s = head_data[head_idx]
    except (IndexError, TypeError): head_s = f"不明:{head_idx}"
    try: pair_s = pair_data[pair_idx]
    except (IndexError, TypeError): pair_s = f"不明:{pair_idx}"

    qr_actions_cell = f"""
    <td class="qr-actions-cell" id="qr-cell-{rsid}"><div class="qr-action-container">
      <div class="qr-placeholder" id="qr-placeholder-{rsid}"
          data-rsid="{rsid}"
          data-qr-str="{qr_str}"
          data-name-s="{name_s}"
          data-clr-s="{clr_s}"
          data-body-s="{body_s}"
          data-ant-s="{ant_s}"
          data-color-idx="{color_idx if color_idx is not None else 0}">
          QR
        </div>
      <img src="" data-qr-id="{rsid}" class="qr-thumb" id="qr-thumb-{rsid}" style="display: none;" onclick="showLargeQr('{rsid}')" title="クリックして拡大表示" alt="{rsid}">
      <div class="button-stack">
        <button class="copy-btn" id="copy-btn-{rsid}" onclick="copyQrToClipboard('{rsid}', this)" disabled>コピー</button>
        <button class="save-btn" id="save-btn-{rsid}" onclick="saveQrImage('{rsid}')" disabled>保存</button>
      </div>
    </div></td>"""
    table_html += f"""
    <tr>
        {qr_actions_cell}
        <td>{rsid}</td>
        <td>{name_s}</td>
        <td>{ant_s}</td>
        <td>{body_s}</td>
        <td>{clr_s}</td>
        <td>{head_s}</td>
        <td>{pair_s}</td>
    </tr>
    """
  table_html += """
  </tbody>
</table>
<div id="qrOverlay" class="qr-overlay" onclick="hideLargeQr()">
  <img id="largeQrImage" src="" onclick="event.stopPropagation()">
</div>

<script>
(function() {
  var qrDataStorage = window.qrDataStorage || {};
  window.qrDataStorage = qrDataStorage;
  const observerOptions = { root: null, rootMargin: '0px', threshold: 0.1 };
  const observerCallback = async (entries, observer) => {
    for (const entry of entries) {
      if (entry.isIntersecting) {
        const placeholder = entry.target;
        const rsid = placeholder.dataset.rsid;
        if (placeholder.classList.contains('loading') || placeholder.classList.contains('loaded')) {
          continue;
        }
        placeholder.classList.add('loading');
        placeholder.textContent = '';
        const qrStr = placeholder.dataset.qrStr;
        const nameS = placeholder.dataset.nameS;
        const clrS = placeholder.dataset.clrS;
        const bodyS = placeholder.dataset.bodyS;
        const antS = placeholder.dataset.antS;
        const colorIdx = placeholder.dataset.colorIdx ? parseInt(placeholder.dataset.colorIdx) : 0;
        try {
          const result = await google.colab.kernel.invokeFunction(
            'generate_qr_for_frontend',
            [rsid, qrStr, nameS, clrS, bodyS, antS, colorIdx],
            {}
          );
          console.debug('invokeFunction response for ' + rsid + ':', result);
          let base64Data = null;
          if (result && result.data) {
            if (typeof result.data['application/json'] === 'string' && result.data['application/json'].startsWith('data:image/png;base64,')) {
              base64Data = result.data['application/json'];
              console.debug('Got base64 from application/json for ' + rsid);
            } else {
              let textPlainData = result.data['text/plain'];
              if (typeof textPlainData === 'string') {
                if (textPlainData.startsWith("'") && textPlainData.endsWith("'")) {
                  textPlainData = textPlainData.substring(1, textPlainData.length - 1);
                  console.debug('Cleaned quotes from text/plain for ' + rsid);
                }
                if (textPlainData.startsWith('data:image/png;base64,')) {
                  base64Data = textPlainData;
                  console.debug('Got base64 from text/plain (after cleanup) for ' + rsid);
                } else {
                  console.warn('text/plain data for ' + rsid + ' does not have valid base64 header after cleanup:', textPlainData);
                }
              } else {
                console.warn('text/plain data is not a string for ' + rsid, textPlainData);
              }
            }
            if (!base64Data) {
              console.warn('Could not extract valid base64 string from result.data for ' + rsid, result.data);
            }
          } else {
            console.warn('No result.data found for ' + rsid, result);
          }
          if (base64Data) {
            qrDataStorage[rsid] = base64Data;
            const thumbImg = document.getElementById(`qr-thumb-${rsid}`);
            thumbImg.src = base64Data;
            thumbImg.style.display = 'block';
            placeholder.style.display = 'none';
            placeholder.classList.remove('loading');
            placeholder.classList.add('loaded');
            document.getElementById(`copy-btn-${rsid}`).disabled = false;
            document.getElementById(`save-btn-${rsid}`).disabled = false;
          } else {
            console.error('QR generation failed for (invalid data format or no data):', rsid);
            placeholder.textContent = 'エラー';
            placeholder.classList.remove('loading');
          }
        } catch (error) {
          console.error('Error during invokeFunction or processing for', rsid, ':', error);
          placeholder.textContent = 'エラー';
          placeholder.classList.remove('loading');
        }
        observer.unobserve(placeholder);
      }
    }
  };
  if (window.qrCodeObserver) {
    console.debug("Disconnecting previous observer");
    window.qrCodeObserver.disconnect();
  }
  const observer = new IntersectionObserver(observerCallback, observerOptions);
  window.qrCodeObserver = observer;
  console.debug("New observer created and observing");
  document.querySelectorAll('.qr-placeholder').forEach(el => {
    observer.observe(el);
  });
  window.showLargeQr = function(rsid) {
    const qrOverlay = document.getElementById('qrOverlay');
    const largeQrImage = document.getElementById('largeQrImage');
    if (qrDataStorage && qrDataStorage[rsid]) {
      largeQrImage.src = qrDataStorage[rsid];
      qrOverlay.style.display = 'flex';
    } else {
      console.error('QR data not found for id:', rsid);
    }
  };

  window.hideLargeQr = function() {
    const qrOverlay = document.getElementById('qrOverlay');
    const largeQrImage = document.getElementById('largeQrImage');
    qrOverlay.style.display = 'none';
    largeQrImage.src = '';
  };

  window.copyQrToClipboard = async function(rsid, buttonElement) {
    const base64Data = qrDataStorage[rsid];
    if (!base64Data) {
      alert('コピーするQRコードデータがありません。');
      return;
    }
    if (!navigator.clipboard || !navigator.clipboard.write) {
        console.warn('Clipboard API (write) not supported or context is insecure.');
        return;
    }

    buttonElement.disabled = true;
    const originalText = buttonElement.textContent;
    buttonElement.textContent = '...';

    try {
      const response = await fetch(base64Data);
      const blob = await response.blob();
      await navigator.clipboard.write([
        new ClipboardItem({
          [blob.type]: blob
        })
      ]);

      buttonElement.textContent = '完了！';
      setTimeout(() => {
          buttonElement.textContent = originalText;
          buttonElement.disabled = false;
      }, 2000);

    } catch (err) {
      console.error('クリップボードへのコピーに失敗しました:', err);
      buttonElement.textContent = 'エラー';
      setTimeout(() => {
          buttonElement.textContent = originalText;
          buttonElement.disabled = false; // ボタンを再度有効化
      }, 2000); // 少し長めに表示
    }
  };
  window.saveQrImage = async function(rsid) {
    const base64Data = qrDataStorage[rsid];
    if (!base64Data) {
      alert('保存するQRコードデータがありません。');
      return;
    }

    try {
      const response = await fetch(base64Data);
      const blob = await response.blob();
      const link = document.createElement('a');
      link.href = URL.createObjectURL(blob);
      const filename = `${rsid}.png`;
      link.download = filename;
      document.body.appendChild(link);
      link.click();
      document.body.removeChild(link);
      URL.revokeObjectURL(link.href);

    } catch (err) {
      console.error('QRコードの保存に失敗しました:', err);
    }
  };
  if (!window.qrEscapeListenerAdded) {
    document.addEventListener('keydown', function(event) {
      if (event.key === "Escape" && document.getElementById('qrOverlay') && document.getElementById('qrOverlay').style.display === 'flex') {
        window.hideLargeQr();
      }
    });
    window.qrEscapeListenerAdded = true;
  }
})();
</script>
"""
  clear_output(wait=True)
  display(HTML(table_html))
print("Done!!")

In [ ]:
#@title 2.データベースを作成(初回)
data=[]
num_processes = os.cpu_count()
with multiprocessing.Pool(processes=num_processes) as pool:
  it = pool.imap_unordered(run,range(256))
  for res in tqdm(it, total=256, desc="Files"):
    if res:
      data.extend(res)
db = sqlite3.connect(db_path)
db.execute("PRAGMA synchronous=OFF;")
db.execute("PRAGMA cache_size = -2000000;")
db.execute("PRAGMA temp_store = MEMORY;")
db.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (rsid TEXT PRIMARY KEY,qr TEXT,name INTEGER,antenna INTEGER,body INTEGER,color INTEGER,depth INTEGER,head INTEGER,pair INTEGER)")
db.commit()
print("Inserting...")
db.executemany(f"INSERT OR IGNORE INTO {table_name} (rsid, qr, name, antenna, body, color, depth, head, pair) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",data)
db.commit()
db.close()
print(f"Inserted {len(data)} records into the database")
del data
print("Done!!")

In [ ]:
#@title ドライブに保存(任意)

from google.colab import drive
import shutil
drive.mount("/content/drive")
shutil.copy2(db_path, "/content/drive/MyDrive/"+db_path)
print("Done!!")

In [ ]:
#@title ドライブから読み込み(保存した場合、2回目以降)

from google.colab import drive
import shutil
drive.mount("/content/drive")
shutil.copy2("/content/drive/MyDrive/"+db_path,db_path)
print("Done!!")

In [ ]:
#@title 3.Scyllaのテスト

print_results(scylla(
  db_path=db_path,
  table_name=table_name,
  rsid=None,
  name="れんじろう",
  antenna="すこしむてき",
  body="1A",
  color=None,
  depth=None,
  head=None,
))

print_results(scylla(
  db_path=db_path,
  table_name=table_name,
  rsid=None,
  name=None,
  antenna="すこしむてき",
  body="1E",
  color="黒色",
  depth=None,
  head="石鹸",
))

print("Done!!")

In [ ]:
#@title 4.Scylla GUI (個体検索)

rsid_widget = widgets.Text(description="RSID:", placeholder='例: aoFMuZ')
name_widget = widgets.Text(description="名前:", placeholder='例: ぼなぱると')
antenna_options = ['指定なし'] + antenna_data
antenna_widget = widgets.Dropdown(options=antenna_options, value='指定なし', description='アンテナ:')
body_options_tuples = ['指定なし'] + body_data
body_widget = widgets.Dropdown(options=body_options_tuples, value='指定なし', description='体格:')
color_options = ['指定なし'] + color_data
color_widget = widgets.Dropdown(options=color_options, value='指定なし', description='色:')
depth_options = ['指定なし', '通常', '濃', '薄']
depth_widget = widgets.Dropdown(options=depth_options, value='指定なし', description='濃淡:')
head_options = ['指定なし'] + head_data
head_widget = widgets.Dropdown(options=head_options, value='指定なし', description='頭の形:')
pair_options = ['指定なし'] + pair_data
pair_widget = widgets.Dropdown(options=pair_options, value='指定なし', description='ペア:')
text_mode_checkbox = widgets.Checkbox(value=True,description='html',disabled=False,indent=False)
search_button = widgets.Button(description="検索実行", button_style='info', icon='search')
results_count_label = widgets.Label(value="")
output_areaS = widgets.Output(layout=widgets.Layout(
  height='480px',
  overflow='scroll',
  border='1px solid #ccc',
  padding='5px'
))

def on_search_button_clicked(b):
  results_count_label.value = "検索中..."
  with output_areaS:
    clear_output(wait=True)
    rsid_val = rsid_widget.value if rsid_widget.value else None
    name_val = name_widget.value if name_widget.value else None
    antenna_val = antenna_widget.value if antenna_widget.value != '指定なし' else None
    body_val = body_widget.value[0:2]  if body_widget.value != '指定なし' else None
    color_val = color_widget.value if color_widget.value != '指定なし' else None
    depth_val = depth_widget.value if depth_widget.value != '指定なし' else None
    head_val = head_widget.value if head_widget.value != '指定なし' else None
    pair_val = pair_widget.value if pair_widget.value != '指定なし' else None
    try:
      results = scylla(
        db_path=db_path,
        table_name=table_name,
        rsid=rsid_val,
        name=name_val,
        antenna=antenna_val,
        body=body_val,
        color=color_val,
        depth=depth_val,
        head=head_val,
        pair=pair_val
      )
      if len(results)==0:
        results_count_label.value = "虚無が見つかりました"
      else:
        results_count_label.value = f"{len(results)} 体見つかりました"
      if text_mode_checkbox.value:
        print_html(results)
      else:
        print_results(results)
    except Exception as e:
      clear_output(wait=True)
      import traceback
      traceback.print_exc()
      results_count_label.value = "エラーが見つかりました"

search_button.on_click(on_search_button_clicked)

gui = widgets.VBox([
  rsid_widget,
  name_widget,
  antenna_widget,
  body_widget,
  color_widget,
  depth_widget,
  head_widget,
  pair_widget,
  widgets.HBox([
    search_button,
    results_count_label,
    text_mode_checkbox,
  ]),
  output_areaS
])

display(gui)

## Bundled tools (テスト中)

In [ ]:
#@title Astraea GUI (QR解析)

!apt-get install -y libzbar0
!pip install pyzbar
import pyzbar.pyzbar as pyzbar

chara_to_qr=[]

def process_img(img):
  decoded_objects = pyzbar.decode(img)
  if decoded_objects:
    charas=[]
    for i, obj in enumerate(decoded_objects):
      hash_object = hashlib.sha256()
      hash_object.update(obj.data + b'\x00')
      h=hash_object.hexdigest()
      for j in range((4 if len(obj.data)<50 else 5 if len(obj.data)<100 else 6)+(1 if h[1] in "13579bdf" else 0)):
        charas.append(int.from_bytes(bytes.fromhex(h[8*j+2:8*j+8])))
    results=[]
    for chara in charas:
      results.extend(get_chara_data("{:06x}".format(chara),chara_to_qr[chara]))
    if html_mode_checkbox.value:
      print_html(results)
    else:
      print_results(results)
  else:
    print("画像内にQRコードが見つかりませんでした。")

def on_upload_change(change):
  with output_areaA:
    clear_output(wait=True)
    if 'new' not in change:
      return
    new_value = change['new']
    uploaded_files_list = []
    if isinstance(new_value, list):
      uploaded_files_list = new_value
    elif isinstance(new_value, dict):
      uploaded_files_list = list(new_value.values())
    else:
      print(f"Error: アップロード情報の形式が不正です。Type: {type(new_value)}")
      return
    if not uploaded_files_list:
        return
    try:
      uploaded_file_info = uploaded_files_list[0]
      if 'content' not in uploaded_file_info:
        raise KeyError("'content' キーがファイル情報に含まれていません。")
      file_content = uploaded_file_info['content']
      try:
        image_stream = io.BytesIO(file_content)
        img = Image.open(image_stream)
        process_img(img)
      except Exception as e:
        print(f"画像処理またはQRデコード中にエラーが発生しました:")
        print(f"詳細: {e}")
        traceback.print_exc()

    except IndexError:
      print("Error: アップロードされたファイルのリストへのアクセスに失敗しました。")
    except KeyError as e:
      print(f"Error: アップロード情報の形式が不正です。キー {e} が見つかりません。")
      if 'uploaded_file_info' in locals():
        info_copy = uploaded_file_info.copy()
        info_copy.pop('content', None)
        print(f"受信データ (content除く): {info_copy}")
    except Exception as e:
      print(f"ファイル情報の取得またはメイン処理中に予期せぬエラーが発生しました:")
      print(f"詳細: {e}")
      traceback.print_exc()

uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='QR画像を選択',
    button_style='primary'
)

output_areaA = widgets.Output(layout=widgets.Layout(
  height='480px',
  overflow='scroll',
  border='1px solid #ccc',
  padding='5px'
))


html_mode_checkbox = widgets.Checkbox(
    value=True,
    description='HTML形式で表示',
    disabled=False,
    indent=False
)

ui = widgets.VBox([
    uploader,
    html_mode_checkbox,
    output_areaA
])
uploader.observe(on_upload_change, names='value')
with output_areaA:
  print("ここにQRコードのデコード結果が表示されます。")

if not chara_to_qr:
  for i in tqdm(range(256), desc="Reading files", unit="file"):
    with open("ndrf_dataset/data/{:02x}.txt".format(i)) as f:
      for l in f:
        l=l.strip()
        if not l:
          continue
        a,b=l.split(" ")
        chara_to_qr.append(b)
clear_output(wait=True)
display(ui)

# 注意事項

ここのQRコードを、対価を得ることを目的として使用することは禁止です。

生成されたQRコードは、Scyllaによって生成されたことがわかるように共有してください。

# Special Thanks
